In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler

In [2]:
import os
os.chdir("C:\\code\\activ")

In [3]:
from programs import lstm_tool

In [4]:
pre_trained_predicter = tf.keras.models.load_model('./model/power_lstm')
checker = tf.keras.models.load_model('./model/checker.h5')

In [5]:
df = pd.read_excel('./documents/nov_nine_var.xlsx').iloc[:,1:22].to_numpy()
scaler = MinMaxScaler()
X = scaler.fit_transform(df)

In [6]:
x, y = lstm_tool.make_sequence(X, 60)

In [7]:
# 예측 -> 확인
# case 0:
#   train
# case 1:
#   recur


In [12]:
y[0]

array([0.94736842, 0.19912861, 0.95490972, 0.16190806, 0.34488938,
       0.30802127, 0.87429939, 0.18766606, 0.49327062, 0.02481508,
       0.98095238, 0.90225564, 0.42261905, 1.        , 0.46268657,
       0.9893617 , 0.14629913, 1.        , 0.05292048, 0.06728395,
       0.07137273])

In [9]:
pre_trained_predicter(x[0:1])

<tf.Tensor: shape=(1, 21), dtype=float32, numpy=
array([[0.9275597 , 0.19049855, 0.94617796, 0.17466497, 0.31856441,
        0.29957438, 0.7031764 , 0.24458782, 0.65632117, 0.03096057,
        0.99207056, 0.9811258 , 0.380858  , 0.9847751 , 0.44820252,
        1.0095489 , 0.13697499, 1.0492946 , 0.08946544, 0.07889979,
        0.07958335]], dtype=float32)>

In [10]:
checker([x[0:1], pre_trained_predicter(x[0:1])])

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[1.]], dtype=float32)>

In [22]:
inp = x[0:1]
for i in range(350):
    with tf.GradientTape() as tape:
        outp = pre_trained_predicter(inp)
        check = checker([inp, outp])

        loss = tf.losses.binary_crossentropy(tf.ones_like(outp), outp)
            
    if int(check) == 0:
        print("update")
        grads = tape.gradient(loss, pre_trained_predicter.trainable_weights)
        pre_trained_predicter.optimizer.apply_gradients(zip(grads, pre_trained_predicter.trainable_weights))

        inp = x[i:i+1]

    if int(check) == 1:
        inp = lstm_tool.shift_data(inp, outp)

    print(i, int(check))

updata
0 0
updata
1 0
updata
2 0
updata
3 0
updata
4 0
updata
5 0
updata
6 0
updata
7 0
updata
8 0


KeyboardInterrupt: 